# Girl Scouts Cookie Booth Locations

## Importing necessary libraries

In [50]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files
import requests

!pip install geopy #install if not already done
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0 #install if not already done
import folium # plotting library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('\nLibraries imported.')

Folium installed

Libraries imported.


## Foursquare credentials

In [2]:
#setting the credentials for Foursquare API
# @hidden
CLIENT_ID = 'NM51MM5XAMQGOSYUTKVYAB3JLIGLG5FMN0BEPF1J15P2GSZG' # your Foursquare ID
CLIENT_SECRET = 'MEQHSISXDAMSXWYAPEF30HBKFLOWPKBBEMKKQPZEHYHFJ04N' # your Foursquare Secret
ACCESS_TOKEN = 'BFXUOFJ2RYT4B5LN2ZIDD2MSCK4KPZPY5DRTDJAWBWWFVI2D' # your FourSquare Access Token

In [3]:
#parameters for Foursquare API
VERSION = '20180604'
LIMIT = 50

## Data Collection and Cleaning

### Download NYC geospatial data

In [4]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [5]:
#extract contents
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

#extracting list of boroughs & neighborhoods
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods_df = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    #neighborhood_name = 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods_df = neighborhoods_df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
    len(neighborhoods_df['Borough'].unique()),
    neighborhoods_df.shape[0]
)
)

neighborhoods_df.head()

The dataframe has 5 boroughs and 306 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Map of New York city

In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [7]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_df['Latitude'], 
                                           neighborhoods_df['Longitude'], 
                                           neighborhoods_df['Borough'], 
                                           neighborhoods_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [8]:
#Unique Neighborhoods in NYC
neighborhoods_df['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

### Filtering on Staten Island borough

In [9]:
staten_df = neighborhoods_df[neighborhoods_df['Borough'] == 'Staten Island'].reset_index(drop=True)
staten_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


### Map of Staten Island

In [10]:
address = 'Staten Island, NY'

geolocator = Nominatim(user_agent="statenisland_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Staten Island are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Staten Island are 40.5834557, -74.1496048.


In [11]:
# create map of Staten Island using latitude and longitude values
map_staten = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(staten_df['Latitude'], staten_df['Longitude'], staten_df['Borough'], staten_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_staten)  
    
map_staten

In [12]:
# Method to call Foursquare API to extract venues for each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500, print_name = False):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        if print_name:
            print('Searching neighborhood of {}'.format(name))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        
        # assign relevant part of JSON to venues
        venues_data = results['response']['venues']        
        
        # tranform venues into a dataframe
        venues = pd.json_normalize(venues_data)        
        
        # keep only columns that include venue name, and anything that is associated with location
        filtered_columns = ['name','categories', 'location.formattedAddress', 'location.lat', 'location.lng']
        venues_filtered = venues.loc[:, filtered_columns]
        # function that extracts the category of the venue
        def get_category_type(row):
            try:
                categories_list = row['categories']
            except:
                categories_list = row['venue.categories']

            if len(categories_list) == 0:
                return None
            else:
                return categories_list[0]['name']

        # filter the category for each row
        venues_filtered['categories'] = venues_filtered.apply(get_category_type, axis=1)
        # clean column names 
        venues_filtered.columns = ['name','categories', 'Address', 'Venue Latitude', 'Venue Longitude']
        venues_filtered['Address'] = venues_filtered['Address'].map(lambda x: x[0])
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v[1][0], 
            v[1][1], 
            v[1][2],
            v[1][3],
            v[1][4]) for v in venues_filtered.iterrows()])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Category', 
                  'Venue Address',
                  'Venue Latitude', 
                  'Venue Longitude']
    
    return(nearby_venues)

In [14]:
#Using the Foursquare API to extract all the venues of Staten Island
all_staten_venues = getNearbyVenues(staten_df["Neighborhood"], 
                                   staten_df['Latitude'], 
                                   staten_df['Longitude'], 
                                   radius=500)

### Neighborhoods of interest

In [15]:
#Filter on towns of interest
towns_of_interest = ['Arrochar','Grasmere', 'Shore Acres', 'Stapleton', 'Concord', 
                     'Old Town', 'Park Hill', 'South Beach']
target_towns_df = staten_df.loc[staten_df['Neighborhood'].isin(towns_of_interest), :]

print('The neighborhoods of interest are {} neighborhoods.\n'.format(len(target_towns_df['Neighborhood'].unique())))
target_towns_df.head(8)

The neighborhoods of interest are 8 neighborhoods.



,Borough,Neighborhood,Latitude,Longitude
2,Staten Island,Stapleton,40.626928,-74.077902
7,Staten Island,South Beach,40.580247,-74.079553
24,Staten Island,Park Hill,40.609190,-74.080157
28,Staten Island,Arrochar,40.596313,-74.067124
29,Staten Island,Grasmere,40.598268,-74.076674
30,Staten Island,Old Town,40.596329,-74.087511
48,Staten Island,Shore Acres,40.609719,-74.066678
50,Staten Island,Concord,40.604473,-74.084024


### Map of my town, Grasmere, Staten Island, NY

In [16]:
address = 'Grasmere, Staten Island, NY'

geolocator = Nominatim(user_agent="grasmere_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Staten Island are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Staten Island are 40.6042711, -74.087086.


In [17]:
# create map of Grasmere, Staten Island using latitude and longitude values
map_grasmere = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(target_towns_df['Latitude'], 
                                           target_towns_df['Longitude'], 
                                           target_towns_df['Borough'], 
                                           target_towns_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_grasmere)  
    
map_grasmere

### Explore neighborhoods of interest

In [18]:
#Using the Foursquare API to extract all the location data of the neighborhoods of interest
staten_venues = getNearbyVenues(target_towns_df["Neighborhood"], 
                                   target_towns_df['Latitude'], 
                                   target_towns_df['Longitude'], 
                                   radius=500,
                                   print_name = True)

Searching neighborhood of Stapleton
Searching neighborhood of South Beach
Searching neighborhood of Park Hill
Searching neighborhood of Arrochar
Searching neighborhood of Grasmere
Searching neighborhood of Old Town
Searching neighborhood of Shore Acres
Searching neighborhood of Concord


In [19]:
staten_venues.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude
0,Stapleton,40.626928,-74.077902,Best Coffee Shop,Coffee Shop,117 Water St (at Wright St),40.627164,-74.077948
1,Stapleton,40.626928,-74.077902,Dollar General,Department Store,135 Canal St (Water Street),40.626935,-74.078271
2,Stapleton,40.626928,-74.077902,Sovereign Bank,Building,Stapleton Branch,40.626982,-74.076977
3,Stapleton,40.626928,-74.077902,Family Dollar,Discount Store,135 Canal St,40.627018,-74.078461
4,Stapleton,40.626928,-74.077902,Defonte's Sandwich Shop,Sandwich Place,95 Water St (Canal Street),40.627202,-74.077257


### Number of venues in Staten Island vs Neighborhoods of Interest

In [20]:
print("Number of venues in Staten Island: {}".format(all_staten_venues.shape[0]))
print("Number of venues in the Neighborhoods of interest: {}".format(staten_venues.shape[0]))

Number of venues in Staten Island: 3150
Number of venues in the Neighborhoods of interest: 400


### Check for missing information

In [21]:
staten_venues.isnull().sum()

Neighborhood               0
Neighborhood Latitude      0
Neighborhood Longitude     0
Venue                      0
Venue Category            66
Venue Address              0
Venue Latitude             0
Venue Longitude            0
dtype: int64

In [36]:
# print(staten_venues.loc[staten_venues['Venue Category'].isna(), :])
# staten_venues.head(48)

In [34]:
#drop rows that have no Venue Category as there is no point if the category is not known
staten_venues.dropna(axis = 0, how = 'any', inplace = True)
staten_venues.reset_index(drop = True, inplace = True)
print("Number of venues in the Neighborhoods of interest: {}".format(staten_venues.shape[0]))


Number of venues in the Neighborhoods of interest: 334


### Unique venue categories in the neighborhoods of interest

In [37]:
print("Number of unique categories in the neighborhoods of interest: {}\n".format(len(staten_venues['Venue Category'].unique())))
print("Unique categories in the neighborhoods of interest:\n {}".format(staten_venues['Venue Category'].unique()))


Number of unique categories in the neighborhoods of interest: 133

Unique categories in the neighborhoods of interest:
 ['Coffee Shop' 'Department Store' 'Building' 'Discount Store'
 'Sandwich Place' 'Pizza Place' 'Asian Restaurant' 'Medical Center'
 'Miscellaneous Shop' 'Park' 'Salon / Barbershop' 'Bus Stop' 'Church'
 'Bus Line' 'Government Building' 'Bank' 'Mexican Restaurant'
 'Cocktail Bar' 'Diner' 'Library' 'Rock Club' 'Train Station'
 'African Restaurant' 'Seafood Restaurant' 'Recording Studio'
 'Mobile Phone Shop' 'Restaurant' 'Sri Lankan Restaurant'
 'Automotive Shop' 'New American Restaurant' 'Tattoo Parlor' 'Lounge'
 'Bar' 'Deli / Bodega' 'Food' 'Hospital' 'Athletics & Sports'
 "Dentist's Office" 'Art Gallery' 'Beach' 'Theme Park' 'Stables'
 'BBQ Joint' 'Pier' 'Gym' 'Skate Park' 'Arts & Crafts Store'
 'Cosmetics Shop' 'Cafeteria' 'Soccer Field'
 'Residential Building (Apartment / Condo)' 'School' "Doctor's Office"
 'College Communications Building' 'Parking' 'Field' 'Playgrou

### Count of venues per neighborhood

In [38]:
staten_venues.groupby(['Neighborhood']).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude
Neighborhood,,,,,,,
Arrochar,46,46,46,46,46,46,46
Concord,42,42,42,42,42,42,42
Grasmere,35,35,35,35,35,35,35
Old Town,46,46,46,46,46,46,46
Park Hill,32,32,32,32,32,32,32
Shore Acres,41,41,41,41,41,41,41
South Beach,43,43,43,43,43,43,43
Stapleton,49,49,49,49,49,49,49


### Analyze each neighborhood

In [39]:
staten_one_hot = pd.get_dummies(staten_venues['Venue Category'], prefix='', prefix_sep='')
staten_one_hot["Neighborhood"] = staten_venues["Neighborhood"]

cols = list(staten_one_hot.columns)
cols = [cols[-1]] + cols[:-1]
staten_one_hot = staten_one_hot[cols]

print(staten_one_hot.shape)

staten_one_hot.head()

(334, 134)


,Neighborhood,African Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Automotive Shop,...,Tattoo Parlor,Tea Room,Temple,Theme Park,Thrift / Vintage Store,Toll Plaza,Train Station,University,Vegetarian / Vegan Restaurant,Voting Booth
0,Stapleton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Stapleton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Stapleton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Stapleton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Stapleton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Top 5 venues for each neighborhood by frequency

In [40]:
#### Determine catergory-wise mean grouping by Neighborhood
staten_grouped = staten_one_hot.groupby(['Neighborhood']).mean().reset_index()
staten_grouped

,Neighborhood,African Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Automotive Shop,...,Tattoo Parlor,Tea Room,Temple,Theme Park,Thrift / Vintage Store,Toll Plaza,Train Station,University,Vegetarian / Vegan Restaurant,Voting Booth
0,Arrochar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
1,Concord,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.047619,...,0.023810,0.00000,0.000000,0.000000,0.047619,0.00000,0.023810,0.000000,0.023810,0.00000
2,Grasmere,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.028571,0.028571,0.00000
3,Old Town,0.000000,0.021739,0.000000,0.000000,0.000000,0.021739,0.000000,0.043478,0.043478,...,0.000000,0.00000,0.021739,0.000000,0.000000,0.00000,0.021739,0.000000,0.000000,0.00000
4,Park Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,...,0.000000,0.03125,0.000000,0.000000,0.031250,0.03125,0.000000,0.000000,0.000000,0.00000
5,Shore Acres,0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.02439
6,South Beach,0.000000,0.000000,0.023256,0.023256,0.000000,0.000000,0.023256,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.023256,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
7,Stapleton,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.020408,...,0.020408,0.00000,0.000000,0.000000,0.000000,0.00000,0.020408,0.000000,0.000000,0.00000


In [41]:
num_venues = 5

for neighbor in staten_grouped['Neighborhood']:
    print('\n----' + neighbor + '----')   
    temp_df = staten_grouped[staten_grouped['Neighborhood'] == neighbor].T.reset_index()
    temp_df.columns = ['venue','freq']
    temp_df = temp_df.iloc[1:]
    temp_df['freq'] = temp_df['freq'].astype(float)
    temp_df = temp_df.round({'freq': 2})
    temp_df.sort_values(by = 'freq', ascending = False, inplace = True)
    temp_df.reset_index(drop = True, inplace = True)
    print(temp_df[:num_venues])


----Arrochar----
                venue  freq
0            Bus Stop  0.07
1       Deli / Bodega  0.07
2  Italian Restaurant  0.04
3         Pizza Place  0.04
4            Bus Line  0.04

----Concord----
                    venue  freq
0                  Office  0.05
1           Deli / Bodega  0.05
2                Building  0.05
3         Doctor's Office  0.05
4  Thrift / Vintage Store  0.05

----Grasmere----
                venue  freq
0            Bus Stop  0.09
1              Church  0.06
2     Doctor's Office  0.06
3  Chinese Restaurant  0.06
4         Pizza Place  0.06

----Old Town----
                venue  freq
0  Italian Restaurant  0.09
1              Bakery  0.07
2                Bank  0.07
3      Medical Center  0.07
4     Auto Dealership  0.04

----Park Hill----
                     venue  freq
0     Other Great Outdoors  0.06
1              Event Space  0.06
2                     Park  0.06
3               Restaurant  0.03
4  Health & Beauty Service  0.03

----Shore Acres

### Segmenting and Clustering neighborhoods to pick the ones that are more similar to my neighborhood

In [42]:
# Function to sort and return most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = staten_grouped['Neighborhood']

for ind in np.arange(staten_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(staten_grouped.iloc[ind, :], num_top_venues)

venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arrochar,Deli / Bodega,Bus Stop,Pizza Place,Bagel Shop,Liquor Store
1,Concord,Doctor's Office,Office,Building,Chinese Restaurant,Thrift / Vintage Store
2,Grasmere,Bus Stop,Chinese Restaurant,Church,Food,Pizza Place
3,Old Town,Italian Restaurant,Medical Center,Bank,Bakery,Pizza Place
4,Park Hill,Park,Other Great Outdoors,Event Space,Bus Line,Church
5,Shore Acres,Italian Restaurant,Deli / Bodega,Chinese Restaurant,Concert Hall,Pizza Place
6,South Beach,Hospital,Beach,Dentist's Office,Gym,Doctor's Office
7,Stapleton,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store


#### Cluster neighborhoods

In [45]:
staten_grouped.head()

,Neighborhood,African Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Automotive Shop,...,Tattoo Parlor,Tea Room,Temple,Theme Park,Thrift / Vintage Store,Toll Plaza,Train Station,University,Vegetarian / Vegan Restaurant,Voting Booth
0,Arrochar,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0
1,Concord,0.0,0.000000,0.0,0.0,0.0,0.000000,0.02381,0.000000,0.047619,...,0.02381,0.00000,0.000000,0.0,0.047619,0.00000,0.023810,0.000000,0.023810,0.0
2,Grasmere,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,...,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.000000,0.028571,0.028571,0.0
3,Old Town,0.0,0.021739,0.0,0.0,0.0,0.021739,0.00000,0.043478,0.043478,...,0.00000,0.00000,0.021739,0.0,0.000000,0.00000,0.021739,0.000000,0.000000,0.0
4,Park Hill,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.031250,...,0.00000,0.03125,0.000000,0.0,0.031250,0.03125,0.000000,0.000000,0.000000,0.0


In [46]:
# set number of clusters
kclusters = 5

cluster_grouped = staten_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cluster_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 4, 0, 1, 2, 3], dtype=int32)

In [47]:
# add clustering labels along with top ten venues 
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1,Arrochar,Deli / Bodega,Bus Stop,Pizza Place,Bagel Shop,Liquor Store
1,0,Concord,Doctor's Office,Office,Building,Chinese Restaurant,Thrift / Vintage Store
2,1,Grasmere,Bus Stop,Chinese Restaurant,Church,Food,Pizza Place
3,4,Old Town,Italian Restaurant,Medical Center,Bank,Bakery,Pizza Place
4,0,Park Hill,Park,Other Great Outdoors,Event Space,Bus Line,Church


In [48]:
staten_merged = staten_venues.copy()

# merge staten venues with venues sorted by neighborhood to add latitude/longitude for each neighborhood
staten_merged = staten_merged.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

staten_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Stapleton,40.626928,-74.077902,Best Coffee Shop,Coffee Shop,117 Water St (at Wright St),40.627164,-74.077948,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
1,Stapleton,40.626928,-74.077902,Dollar General,Department Store,135 Canal St (Water Street),40.626935,-74.078271,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
2,Stapleton,40.626928,-74.077902,Sovereign Bank,Building,Stapleton Branch,40.626982,-74.076977,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
3,Stapleton,40.626928,-74.077902,Family Dollar,Discount Store,135 Canal St,40.627018,-74.078461,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
4,Stapleton,40.626928,-74.077902,Defonte's Sandwich Shop,Sandwich Place,95 Water St (Canal Street),40.627202,-74.077257,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store


#### Visualizing clusters of neighborhood

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(staten_merged['Neighborhood Latitude'], staten_merged['Neighborhood Longitude'], staten_merged['Neighborhood'], staten_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining Clusters

In [ ]:
#Picking the necessary columns 
cols = staten_merged.columns.tolist()
cols = [cols[2]] + cols[5:staten_merged.shape[1]]

### Examining the cluster that my neighborhood falls in

In [55]:
staten_merged

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Stapleton,40.626928,-74.077902,Best Coffee Shop,Coffee Shop,117 Water St (at Wright St),40.627164,-74.077948,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
1,Stapleton,40.626928,-74.077902,Dollar General,Department Store,135 Canal St (Water Street),40.626935,-74.078271,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
2,Stapleton,40.626928,-74.077902,Sovereign Bank,Building,Stapleton Branch,40.626982,-74.076977,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
3,Stapleton,40.626928,-74.077902,Family Dollar,Discount Store,135 Canal St,40.627018,-74.078461,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
4,Stapleton,40.626928,-74.077902,Defonte's Sandwich Shop,Sandwich Place,95 Water St (Canal Street),40.627202,-74.077257,3,Pizza Place,Bank,Church,Salon / Barbershop,Discount Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,Concord,40.604473,-74.084024,Pain Management,Doctor's Office,2071 Clove road (Mosel Avenue),40.604973,-74.083160,0,Doctor's Office,Office,Building,Chinese Restaurant,Thrift / Vintage Store
330,Concord,40.604473,-74.084024,Bubbles Playhouse,Event Space,2012 Clove rd,40.604798,-74.084759,0,Doctor's Office,Office,Building,Chinese Restaurant,Thrift / Vintage Store
331,Concord,40.604473,-74.084024,Xin Chinese Japanese,Chinese Restaurant,2071 Clove Rd,40.604876,-74.083043,0,Doctor's Office,Office,Building,Chinese Restaurant,Thrift / Vintage Store
332,Concord,40.604473,-74.084024,Richmond Imaging,Building,2071,40.604876,-74.083043,0,Doctor's Office,Office,Building,Chinese Restaurant,Thrift / Vintage Store


In [57]:
staten_merged.loc[staten_merged['Cluster Labels'] == 1, :].head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
124,Arrochar,40.596313,-74.067124,MTA Bus - S53,Bus Line,"Staten Island, NY",40.598965,-74.063835,1,Deli / Bodega,Bus Stop,Pizza Place,Bagel Shop,Liquor Store
125,Arrochar,40.596313,-74.067124,McClean Cleaners,Dry Cleaner,134 McClean Ave,40.597934,-74.066354,1,Deli / Bodega,Bus Stop,Pizza Place,Bagel Shop,Liquor Store
126,Arrochar,40.596313,-74.067124,Diggie's Vomitorium,Restaurant,Oh this is gross and smelly,40.594499,-74.068011,1,Deli / Bodega,Bus Stop,Pizza Place,Bagel Shop,Liquor Store
127,Arrochar,40.596313,-74.067124,Via Ponte Pizzeria,Pizza Place,147 McClean Ave,40.598046,-74.067080,1,Deli / Bodega,Bus Stop,Pizza Place,Bagel Shop,Liquor Store
128,Arrochar,40.596313,-74.067124,MTA Bus - MC Clean Av & Railroad Av (S53),Bus Stop,McClean Ave (at Conger St),40.598143,-74.066233,1,Deli / Bodega,Bus Stop,Pizza Place,Bagel Shop,Liquor Store


### Filtering neighborhoods to match my neighborhood

In [65]:
towns_of_interest = staten_merged.loc[staten_merged['Cluster Labels'] == 1, 'Neighborhood'].unique()
towns_of_interest

array(['Arrochar', 'Grasmere', 'Shore Acres'], dtype=object)

In [84]:
#Filter on towns of interest
# towns_of_interest = ['Arrochar','Grasmere', 'Shore Acres', 'Stapleton', 'Concord', 
#                      'Old Town', 'Park Hill', 'South Beach']
target_towns = staten_venues.loc[staten_venues['Neighborhood'].isin(towns_of_interest), :]
target_towns.reset_index(drop = True, inplace = True)
print('The neighborhoods of interest are {} neighborhoods.\n'.format(len(target_towns['Neighborhood'].unique())))
print('The neighborhoods of interest are {}\n'.format(target_towns['Neighborhood'].unique()))

target_towns.head(8)

The neighborhoods of interest are 3 neighborhoods.

The neighborhoods of interest are ['Arrochar' 'Grasmere' 'Shore Acres']



,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude
0,Arrochar,40.596313,-74.067124,MTA Bus - S53,Bus Line,"Staten Island, NY",40.598965,-74.063835
1,Arrochar,40.596313,-74.067124,McClean Cleaners,Dry Cleaner,134 McClean Ave,40.597934,-74.066354
2,Arrochar,40.596313,-74.067124,Diggie's Vomitorium,Restaurant,Oh this is gross and smelly,40.594499,-74.068011
3,Arrochar,40.596313,-74.067124,Via Ponte Pizzeria,Pizza Place,147 McClean Ave,40.598046,-74.067080
4,Arrochar,40.596313,-74.067124,MTA Bus - MC Clean Av & Railroad Av (S53),Bus Stop,McClean Ave (at Conger St),40.598143,-74.066233
5,Arrochar,40.596313,-74.067124,Ocean Deli,Deli / Bodega,ocean (robin),40.596449,-74.068488
6,Arrochar,40.596313,-74.067124,South Beach Boardwalk,Beach,Father Capodanno Blvd (Sand Ln),40.590215,-74.066033
7,Arrochar,40.596313,-74.067124,J&R Pizza,Pizza Place,133 McClean Ave (at Hastings St),40.598225,-74.066566


### Foursquare Recovery Index - Foot traffic 

In [67]:
#Extract top 20 categories from the Foursquare Recovery Index
foot_traffic_df = pd.read_csv("Foursquare_Recovey_Index.csv", delimiter = ',')
foot_traffic_df.head(5)

,Date,Category_1,Category_2,Category_3,Category_4,Category_5,Category_6,Category_7,Category_8,Category_9,...,Category_11,Category_12,Category_13,Category_14,Category_15,Category_16,Category_17,Category_18,Category_19,Category_20
0,2/8/21,Beaches,Outdoors,Bank,Skiing,Metro,Medical,Auto Shops,Bars,Transit,...,School,"Liquor, Beer, and Wine Stores",Warehouse Store,Sports,Convenience Stores,Government,Hotels,Discount,Grocery,Bus
1,2/9/21,Beaches,Outdoors,Bank,Skiing,Metro,Medical,Auto Shops,Bars,Transit,...,School,"Liquor, Beer, and Wine Stores",Warehouse Store,Sports,Convenience Stores,Government,Hotels,Discount,Grocery,Bus
2,2/10/21,Beaches,Outdoors,Bank,Skiing,Metro,Medical,Auto Shops,Bars,Transit,...,School,"Liquor, Beer, and Wine Stores",Warehouse Store,Sports,Convenience Stores,Government,Hotels,Discount,Grocery,Bus
3,2/11/21,Beaches,Outdoors,Bank,Skiing,Metro,Medical,Auto Shops,Bars,Transit,...,School,"Liquor, Beer, and Wine Stores",Warehouse Store,Sports,Convenience Stores,Government,Hotels,Discount,Grocery,Bus
4,2/12/21,Airports,School,Bus,Pharmacies and Drug Stores,Transit,Metro,Apparel Retail,Office,Discount,...,Fitness,Big Box Store,Warehouse Store,Restaurant,Grocery,Bars,Home Stores and Services,Spiritual Centers,Salons and Services,Hotels


In [68]:
print("Number of unique categories in the Recovery Index: {}\n".format(len(np.unique(foot_traffic_df.iloc[:, 1:]))))
print("Unique categories in the Recovery Index:\n {}".format(np.unique(foot_traffic_df.iloc[:, 1:])))


Number of unique categories in the Recovery Index: 34

Unique categories in the Recovery Index:
 ['Airports' 'Apparel Retail' 'Auto DealerShips' 'Auto Shops' 'Bank' 'Bars'
 'Beaches' 'Big Box Store' 'Bus' 'Convenience Stores' 'Discount'
 'Fast Food' 'Fitness' 'Gas' 'Government' 'Grocery' 'Hardware'
 'Home Stores and Services' 'Hotels' 'Liquor, Beer, and Wine Stores'
 'Medical' 'Metro' 'Office' 'Outdoors' 'Pet Stores'
 'Pharmacies and Drug Stores' 'Restaurant' 'Salons and Services' 'School'
 'Skiing' 'Spiritual Centers' 'Sports' 'Transit' 'Warehouse Store']


In [69]:
traffic_df = foot_traffic_df.copy()
cols = ['FootTrafficDate']
foottraffic_rank_df = pd.DataFrame(columns = cols)
#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in foot_traffic_df.iterrows():
    foottraffic_rank_df.at[index, 'FootTrafficDate'] = row['Date']
    for idx in range(1,21):
        foottraffic_rank_df.at[index, row['Category_' + str(idx)]] = idx #(20 - idx) + 1

foottraffic_rank_df['FootTrafficDate'] = foottraffic_rank_df['FootTrafficDate'].astype('datetime64[ns]')

#Set the NaNs to 21 to indicate for categories with no rank
foottraffic_rank_df = foottraffic_rank_df.fillna(21)
foottraffic_rank_df.head()

,FootTrafficDate,Beaches,Outdoors,Bank,Skiing,Metro,Medical,Auto Shops,Bars,Transit,...,Big Box Store,Restaurant,Home Stores and Services,Spiritual Centers,Salons and Services,Pet Stores,Hardware,Fast Food,Auto DealerShips,Gas
0,2021-02-08,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0
1,2021-02-09,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0
2,2021-02-10,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0
3,2021-02-11,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0
4,2021-02-12,21.0,21.0,21.0,21.0,6.0,10.0,21.0,16.0,5.0,...,12.0,14.0,17.0,18.0,19.0,21.0,21.0,21.0,21.0,21.0


In [70]:
num_venues = 20
print("")
print("--- Top 20 categories based on Foursquare Recovey Index ---")

fsr_rank_df = foottraffic_rank_df.mean(numeric_only = True).sort_values(ascending = True).reset_index()
fsr_rank_df.columns = ['Venue Category','Ranking']

fsr_rank_df[:num_venues]


--- Top 20 categories based on Foursquare Recovey Index ---


,Venue Category,Ranking
0,Airports,7.705882
1,Skiing,9.941176
2,Bars,10.000000
3,Apparel Retail,10.529412
4,Beaches,11.882353
5,Discount,12.294118
6,Hotels,12.882353
7,Bank,13.058824
8,Metro,13.529412
9,Medical,13.588235


### Fixing the Venue categories to match the Generic Foursquare Recovey Index categories

In [85]:
target_towns.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude
0,Arrochar,40.596313,-74.067124,MTA Bus - S53,Bus Line,"Staten Island, NY",40.598965,-74.063835
1,Arrochar,40.596313,-74.067124,McClean Cleaners,Dry Cleaner,134 McClean Ave,40.597934,-74.066354
2,Arrochar,40.596313,-74.067124,Diggie's Vomitorium,Restaurant,Oh this is gross and smelly,40.594499,-74.068011
3,Arrochar,40.596313,-74.067124,Via Ponte Pizzeria,Pizza Place,147 McClean Ave,40.598046,-74.067080
4,Arrochar,40.596313,-74.067124,MTA Bus - MC Clean Av & Railroad Av (S53),Bus Stop,McClean Ave (at Conger St),40.598143,-74.066233


In [86]:
print("Number of unique categories in the neighborhoods of interest: {}\n".format(len(target_towns['Venue Category'].unique())))
print("Unique categories in the neighborhoods of interest:\n {}".format(target_towns['Venue Category'].unique()))

Number of unique categories in the neighborhoods of interest: 64

Unique categories in the neighborhoods of interest:
 ['Bus Line' 'Dry Cleaner' 'Restaurant' 'Pizza Place' 'Bus Stop'
 'Deli / Bodega' 'Beach' 'Liquor Store' 'Italian Restaurant'
 'Middle Eastern Restaurant' 'Social Club' 'Outdoors & Recreation'
 'Playground' 'Salon / Barbershop' 'Polish Restaurant' 'Event Space'
 'Chinese Restaurant' 'Bagel Shop' 'Supermarket' 'Taco Place'
 'Dance Studio' 'High School' 'Mediterranean Restaurant'
 'Mobile Phone Shop' 'Sporting Goods Shop' "Doctor's Office"
 'Funeral Home' 'Food' 'Laundry Service' 'Sculpture Garden'
 'Elementary School' 'Bar' 'Bus Station' 'Ice Cream Shop' 'Bank'
 'Japanese Restaurant' 'University' 'Pharmacy' 'Courthouse' 'Office'
 'Steakhouse' 'Grocery Store' 'Spa' 'Gym / Fitness Center'
 'Latin American Restaurant' 'Nightlife Spot'
 'Vegetarian / Vegan Restaurant' 'Church' 'Rest Area' "Dentist's Office"
 'Gastropub' 'Art Gallery' 'Café' 'Pawn Shop' 'Bike Shop' 'Music Ven

In [88]:
target_df = target_towns.copy()
target_df["Venue Category Orig"] = target_towns['Venue Category']
target_towns = target_df
target_towns.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
0,Arrochar,40.596313,-74.067124,MTA Bus - S53,Bus Line,"Staten Island, NY",40.598965,-74.063835,Bus Line
1,Arrochar,40.596313,-74.067124,McClean Cleaners,Dry Cleaner,134 McClean Ave,40.597934,-74.066354,Dry Cleaner
2,Arrochar,40.596313,-74.067124,Diggie's Vomitorium,Restaurant,Oh this is gross and smelly,40.594499,-74.068011,Restaurant
3,Arrochar,40.596313,-74.067124,Via Ponte Pizzeria,Pizza Place,147 McClean Ave,40.598046,-74.067080,Pizza Place
4,Arrochar,40.596313,-74.067124,MTA Bus - MC Clean Av & Railroad Av (S53),Bus Stop,McClean Ave (at Conger St),40.598143,-74.066233,Bus Stop


**Restaurants:**
Exploring restaurants, it is clear that the venues are by cuisine. 

In [89]:
target_towns.loc[target_towns['Venue Category'].str.contains('Restaurant'), :].head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
2,Arrochar,40.596313,-74.067124,Diggie's Vomitorium,Restaurant,Oh this is gross and smelly,40.594499,-74.068011,Restaurant
9,Arrochar,40.596313,-74.067124,Italianissimo,Italian Restaurant,107 McClean Ave,40.598596,-74.065619,Italian Restaurant
10,Arrochar,40.596313,-74.067124,Basilio's Inn,Italian Restaurant,6 Galesville Ct,40.597369,-74.063674,Italian Restaurant
11,Arrochar,40.596313,-74.067124,Chinar On The Island,Middle Eastern Restaurant,283 Sand Ln,40.592726,-74.067994,Middle Eastern Restaurant
19,Arrochar,40.596313,-74.067124,Terminal 1,Polish Restaurant,323 Sand Ln,40.592107,-74.067857,Polish Restaurant


We are only interested in generic venue category and not cuisines, so we clean up by setting venue categories as Restaurants

In [90]:
#Cleaning up Restaurant venue category
target_towns.loc[target_towns['Venue Category'].str.contains('Restaurant'), 'Venue Category'] = 'Restaurant'
target_towns.loc[target_towns['Venue Category'].str.contains('Restaurant'), :].head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
2,Arrochar,40.596313,-74.067124,Diggie's Vomitorium,Restaurant,Oh this is gross and smelly,40.594499,-74.068011,Restaurant
9,Arrochar,40.596313,-74.067124,Italianissimo,Restaurant,107 McClean Ave,40.598596,-74.065619,Italian Restaurant
10,Arrochar,40.596313,-74.067124,Basilio's Inn,Restaurant,6 Galesville Ct,40.597369,-74.063674,Italian Restaurant
11,Arrochar,40.596313,-74.067124,Chinar On The Island,Restaurant,283 Sand Ln,40.592726,-74.067994,Middle Eastern Restaurant
19,Arrochar,40.596313,-74.067124,Terminal 1,Restaurant,323 Sand Ln,40.592107,-74.067857,Polish Restaurant


**Sports venues:**
Looking for venues that are Sports related

In [93]:
target_towns.loc[target_towns['Venue Category'].isin(['Sporting Goods Shop', 'Athletics & Sports']), :]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
33,Arrochar,40.596313,-74.067124,Treshers,Sporting Goods Shop,186 Robin Rd (Sand Lane),40.592519,-74.067604,Sporting Goods Shop


The above rows can be grouped under the Sports category

In [94]:
#Cleaning up Sports venue category
target_towns.loc[target_towns['Venue Category'].isin(['Sporting Goods Shop', 'Athletics & Sports']),
                                                        'Venue Category'] = 'Sports'
target_towns.loc[target_towns['Venue Category Orig'].isin(['Sporting Goods Shop', 'Athletics & Sports']),:]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
33,Arrochar,40.596313,-74.067124,Treshers,Sports,186 Robin Rd (Sand Lane),40.592519,-74.067604,Sporting Goods Shop


**Spiritual Centers:**
Looking for venues that are Spiritual Centers

In [95]:
#Cleaning up Spiritual Centers venue category
target_towns.loc[target_towns['Venue Category'].isin(['Church', 'Synagogue', 'Spiritual Center' ]),
                                                        'Venue Category'] = 'Spiritual Centers'
target_towns.loc[target_towns['Venue Category Orig'].isin(['Church', 'Synagogue', 'Spiritual Center' ]),:]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
73,Grasmere,40.598268,-74.076674,Knights Of Columbus Madonna Council,Spiritual Centers,"Staten Island, NY 10305",40.596131,-74.075866,Church
78,Grasmere,40.598268,-74.076674,Holy Rosary R.C. Church,Spiritual Centers,80 Jerome Ave,40.595345,-74.075933,Church
96,Shore Acres,40.609719,-74.066678,Staten Island Mar Thoma Church,Spiritual Centers,Faber St,40.611977,-74.068203,Church
107,Shore Acres,40.609719,-74.066678,St. John's Episcopal Church,Spiritual Centers,1333 Bay St (btwn New Ln & Anchor Pl),40.611493,-74.063598,Church


**Outdoor venues:**
Looking for venues that are outdoors and consolidating the category as Outdoors

In [96]:
#Cleaning up Outdoors venue category
target_towns.loc[target_towns['Venue Category'].isin(['Park', 'Playground', 'Outdoors & Recreation',
                                                        'Sculpture Garden', 'Rest Area', 'Lake', 
                                                        'Dog Run', 'Other Great Outdoors', 'Scenic Lookout',
                                                       'Performing Arts Venue', 'Art Gallery' , 'Concert Hall']),
                                                        'Venue Category'] = 'Outdoors'

target_towns.loc[target_towns['Venue Category'].str.contains('Outdoors'), :].head()                                                        

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
13,Arrochar,40.596313,-74.067124,Gateway National Recreation Area,Outdoors,"Staten Island, NY 10305",40.592428,-74.065373,Outdoors & Recreation
15,Arrochar,40.596313,-74.067124,Swing Life Away Swings,Outdoors,71 Sand Ln,40.597928,-74.070138,Playground
39,Arrochar,40.596313,-74.067124,The monument,Outdoors,115 lily pond ave,40.600412,-74.064524,Sculpture Garden
76,Grasmere,40.598268,-74.076674,Holy Rosary Parking Lot,Outdoors,"Staten Island, NY 10305",40.595889,-74.076422,Rest Area
85,Shore Acres,40.609719,-74.066678,Poolside At The Dawson's,Outdoors,New York,40.610948,-74.066410,Art Gallery


**Bus:** Consolidating the category, Bus

In [97]:
#Cleaning up Bus venue category
target_towns.loc[target_towns['Venue Category'].isin(['Bus Stop', 'Bus Station', 'Bus Line']),
                                                        'Venue Category'] = 'Bus'

target_towns.loc[target_towns['Venue Category Orig'].isin(['Bus Stop', 'Bus Station', 'Bus Line']), :].head()                                                        

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
0,Arrochar,40.596313,-74.067124,MTA Bus - S53,Bus,"Staten Island, NY",40.598965,-74.063835,Bus Line
4,Arrochar,40.596313,-74.067124,MTA Bus - MC Clean Av & Railroad Av (S53),Bus,McClean Ave (at Conger St),40.598143,-74.066233,Bus Stop
16,Arrochar,40.596313,-74.067124,S53 - Bay Ridge,Bus,"Staten Island, NY 10305",40.597251,-74.068545,Bus Line
26,Arrochar,40.596313,-74.067124,MTA Bus - Lily Pond Av & McClean Av (S51/S53/X...,Bus,Lily Pond Ave (at McClean Av),40.598814,-74.063710,Bus Stop
30,Arrochar,40.596313,-74.067124,MTA Bus - Father Capodanno Blvd & Ocean Ave (S...,Bus,Father Capodonno Blvd (Ocean Ave),40.595101,-74.062776,Bus Stop


**Category : Bars and Liquor, Beer, and Wine Stores**

In [105]:
#Cleaning up Bars category
target_towns.loc[target_towns['Venue Category'].isin(['Cocktail Bar', 'Gastropub', 'Bar', 'Nightlife Spot',
                                                        'Other Nightlife']),
                                                        'Venue Category'] = 'Bars'

#Cleaning up Liquor, Beer, and Wine Stores  category
target_towns.loc[target_towns['Venue Category'].isin(['Liquor Store']),
                                                        'Venue Category'] = 'Liquor, Beer, and Wine Stores'

target_towns.loc[target_towns['Venue Category Orig'].isin(['Cocktail Bar', 'Gastropub', 'Bar', 
                                                           'Nightlife Spot','Liquor Store']), :]


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
8,Arrochar,40.596313,-74.067124,Arrochar,"Liquor, Beer, and Wine Stores",103 McClean Ave,40.594412,-74.068013,Liquor Store
23,Arrochar,40.596313,-74.067124,South Beach Wines And Liquors,"Liquor, Beer, and Wine Stores",325 Sand Ln (at Quincy Ave),40.592030,-74.067654,Liquor Store
41,Arrochar,40.596313,-74.067124,Surf Club,Bars,124 Ocean Ave,40.595364,-74.063171,Bar
70,Grasmere,40.598268,-74.076674,The Basement At 649,Bars,New York,40.603423,-74.075694,Nightlife Spot
71,Grasmere,40.598268,-74.076674,the trailer,Bars,"Staten Island, NY 10305",40.593456,-74.077972,Bar
82,Shore Acres,40.609719,-74.066678,The Phunky Elephant Gastropub,Bars,1271 Bay St (Scarboro Ave),40.612489,-74.064894,Gastropub
94,Shore Acres,40.609719,-74.066678,Danny Blaines,Bars,1384 Bay St,40.609820,-74.063232,Bar
116,Shore Acres,40.609719,-74.066678,Rosebank Tavern,Bars,1258 Bay St,40.612474,-74.065212,Bar


Similary, cleaning up the rest of the categories to reflect the generic ones in the Foursquare recovery index

In [102]:
#Cleaning up Salons and Services category
target_towns.loc[target_towns['Venue Category'].isin(['Salon / Barbershop', 'Spa', 'Nail Salon', 'Health & Beauty Service',
                                                       'Tanning Salon', ]),
                                                        'Venue Category'] = 'Salons and Services'

#Cleaning up Restaurant category
target_towns.loc[target_towns['Venue Category'].isin(['Diner', 'Steakhouse']),
                                                        'Venue Category'] = 'Restaurant'

#Cleaning up Office category
target_towns.loc[target_towns['Venue Category'].isin(['Building', 'Recording Studio', 'Rock Club',
                                                       'Office', "Doctor's Office", "Dentist's Office",
                                                       'Financial or Legal Service']),
                                                        'Venue Category'] = 'Office'

#Cleaning up Fast Food category
target_towns.loc[target_towns['Venue Category'].isin(['Coffee Shop', 'Sandwich Place', 'Pizza Place',
                                                        'Deli / Bodega', 'Taco Place', 'Bagel Shop',
                                                        'Ice Cream Shop', 'Food & Drink Shop',
                                                        'Café', 'Bakery', 'Breakfast Spot',
                                                        'Fried Chicken Joint']),
                                                        'Venue Category'] = 'Fast Food'

#Cleaning up Bank category
target_towns.loc[target_towns['Venue Category'].isin(['Check Cashing Service', 'Bank']),
                                                        'Venue Category'] = 'Bank'


#Cleaning up Home Stores and Services category
target_towns.loc[target_towns['Venue Category'].isin(['Dry Cleaner', 'Tattoo Parlor', 'Dance Studio', 
                                                        'Pawn Shop', 'Social Club', 'Laundry Service', 'Music Store',
                                                        'Library', 'Music Venue', 'Bike Shop', 'Funeral Home', 'Event Space',
                                                        'Furniture / Home Store', 'General Entertainment']),
                                                        'Venue Category'] = 'Home Stores and Services'

#Cleaning up School category
target_towns.loc[target_towns['Venue Category'].isin(['High School', 'University', 'Elementary School',
                                                       'Nursery School', 'Martial Arts School', 'Daycare']),
                                                        'Venue Category'] = 'School'

#Cleaning up Government category
target_towns.loc[target_towns['Venue Category'].isin(['Government Building', 'Courthouse', 'Voting Booth' ]),
                                                        'Venue Category'] = 'Government'

#Cleaning up Medical category
target_towns.loc[target_towns['Venue Category'].isin(['Medical Center']),
                                                        'Venue Category'] = 'Medical'

#Cleaning up Pharmacies and Drug Stores category
target_towns.loc[target_towns['Venue Category'].isin(['Pharmacy']),
                                                        'Venue Category'] = 'Pharmacies and Drug Stores'

#Cleaning up Discount category
target_towns.loc[target_towns['Venue Category'].isin(['Discount Store']),
                                                        'Venue Category'] = 'Discount'

#Cleaning up Apparel Retail category
target_towns.loc[target_towns['Venue Category'].isin(['Department Store', 'Boutique', "Women's Store"]),
                                                        'Venue Category'] = 'Apparel Retail'

#Cleaning up Fitness category
target_towns.loc[target_towns['Venue Category'].isin(['Gym / Fitness Center']),
                                                        'Venue Category'] = 'Fitness'

#Cleaning up Grocery category
target_towns.loc[target_towns['Venue Category'].isin(['Grocery Store', 'Supermarket']),
                                                       'Venue Category'] = 'Grocery'

#Cleaning up Transit category
target_towns.loc[target_towns['Venue Category'].isin(['Train Station']),
                                                       'Venue Category'] = 'Transit'

#Cleaning up Auto DealerShips category
target_towns.loc[target_towns['Venue Category'].isin(['Auto Dealership']),
                                                       'Venue Category'] = 'Auto DealerShips'

#Cleaning up Auto Shops category
target_towns.loc[target_towns['Venue Category'].isin(['Automotive Shop'  ]),
                                                       'Venue Category'] = 'Auto Shops'

#Cleaning up Hardware category
target_towns.loc[target_towns['Venue Category'].isin(['Hardware Store']),
                                                       'Venue Category'] = 'Hardware'

#Cleaning up Convenience Stores category
target_towns.loc[target_towns['Venue Category'].isin(['Mobile Phone Shop', 'Miscellaneous Shop',
                                                        'Cosmetics Shop', 'Thrift / Vintage Store']),
                                                        'Venue Category'] = 'Convenience Stores'

#Cleaning up Pet Stores category
target_towns.loc[target_towns['Venue Category'].isin(['Pet Service']),
                                                        'Venue Category'] = 'Pet Stores'



In [106]:
print("Number of unique categories in the neighborhoods of interest: {}\n".format(len(target_towns['Venue Category'].unique())))
print("Unique categories in the neighborhoods of interest:\n {}".format(target_towns['Venue Category'].unique()))


Number of unique categories in the neighborhoods of interest: 21

Unique categories in the neighborhoods of interest:
 ['Bus' 'Home Stores and Services' 'Restaurant' 'Fast Food' 'Beach'
 'Liquor, Beer, and Wine Stores' 'Outdoors' 'Salons and Services'
 'Grocery' 'School' 'Convenience Stores' 'Sports' 'Office' 'Food' 'Bars'
 'Bank' 'Pharmacies and Drug Stores' 'Government' 'Fitness'
 'Spiritual Centers' 'Hardware']


## Top 10 locations to set up Girl Scouts Cookie Booth locations

In [107]:
# Foursquare Recovery Index
fsr_rank_df

,Venue Category,Ranking
0,Airports,7.705882
1,Skiing,9.941176
2,Bars,10.000000
3,Apparel Retail,10.529412
4,Beaches,11.882353
5,Discount,12.294118
6,Hotels,12.882353
7,Bank,13.058824
8,Metro,13.529412
9,Medical,13.588235


In [108]:
print(target_towns.shape)
target_towns.head()

(122, 9)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig
0,Arrochar,40.596313,-74.067124,MTA Bus - S53,Bus,"Staten Island, NY",40.598965,-74.063835,Bus Line
1,Arrochar,40.596313,-74.067124,McClean Cleaners,Home Stores and Services,134 McClean Ave,40.597934,-74.066354,Dry Cleaner
2,Arrochar,40.596313,-74.067124,Diggie's Vomitorium,Restaurant,Oh this is gross and smelly,40.594499,-74.068011,Restaurant
3,Arrochar,40.596313,-74.067124,Via Ponte Pizzeria,Fast Food,147 McClean Ave,40.598046,-74.067080,Pizza Place
4,Arrochar,40.596313,-74.067124,MTA Bus - MC Clean Av & Railroad Av (S53),Bus,McClean Ave (at Conger St),40.598143,-74.066233,Bus Stop


#### Sort by Foursquare Recovery Index - Foot Traffic

In [114]:
ranked_venues = target_towns.merge(fsr_rank_df, on = 'Venue Category')
ranked_venues.sort_values(by = ['Ranking'], ascending = True, inplace = True)

In [116]:
ranked_venues[['Venue', 'Venue Category', 'Venue Address', 'Venue Latitude', 'Venue Longitude', 'Ranking']].head(8)

,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Ranking
102,Danny Blaines,Bars,1384 Bay St,40.609820,-74.063232,10.000000
101,The Phunky Elephant Gastropub,Bars,1271 Bay St (Scarboro Ave),40.612489,-74.064894,10.000000
100,the trailer,Bars,"Staten Island, NY 10305",40.593456,-74.077972,10.000000
99,The Basement At 649,Bars,New York,40.603423,-74.075694,10.000000
98,Surf Club,Bars,124 Ocean Ave,40.595364,-74.063171,10.000000
103,Rosebank Tavern,Bars,1258 Bay St,40.612474,-74.065212,10.000000
104,"Richmond County Savings Bank, a division of Ne...",Bank,1100 Hylan Blvd,40.598318,-74.078908,13.058824
79,St. Joseph Hill Academy,School,850 Hylan Blvd,40.599649,-74.071011,13.588235


As Bars were kept in the dataset just out of interest, it is obvious that the bars is not a place to put up a Girl Scouts Booth so removing the Bars from the dataset.

In [119]:
ranked_venues = ranked_venues[ranked_venues.loc[:, 'Venue Category'] != 'Bars'].reset_index(drop=True)

In [120]:
top10_venues = ranked_venues[:10]
top10_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Address,Venue Latitude,Venue Longitude,Venue Category Orig,Ranking
0,Grasmere,40.598268,-74.076674,"Richmond County Savings Bank, a division of Ne...",Bank,1100 Hylan Blvd,40.598318,-74.078908,Bank,13.058824
1,Arrochar,40.596313,-74.067124,St. Joseph Hill Academy,School,850 Hylan Blvd,40.599649,-74.071011,High School,13.588235
2,Grasmere,40.598268,-74.076674,Saint Joseph Hill,School,"850 Hylan Blvd, Staten Island, NY 10305",40.598146,-74.070645,University,13.588235
3,Shore Acres,40.609719,-74.066678,Kidz Corner Daycare,School,"Staten Island, NY 10305",40.610466,-74.067023,Daycare,13.588235
4,Arrochar,40.596313,-74.067124,P.S. 39 Francis J. Murphy Jr.,School,71 Sand Ln (btwn Major & MacFarland Ave),40.597828,-74.071243,Elementary School,13.588235
5,Shore Acres,40.609719,-74.066678,Soldivera Bob Orchestra,Outdoors,245 Saint Johns Ave,40.608963,-74.071812,Concert Hall,13.764706
6,Shore Acres,40.609719,-74.066678,White playground,Outdoors,Lyman ave,40.607167,-74.063016,Playground,13.764706
7,Shore Acres,40.609719,-74.066678,Bam Square,Outdoors,Bay St. (bet Hope Ave & Bel Air Rd),40.610760,-74.064803,Dog Run,13.764706
8,Shore Acres,40.609719,-74.066678,Poolside At The Dawson's,Outdoors,New York,40.610948,-74.066410,Art Gallery,13.764706
9,Grasmere,40.598268,-74.076674,Holy Rosary Parking Lot,Outdoors,"Staten Island, NY 10305",40.595889,-74.076422,Rest Area,13.764706


In [121]:
# create map of Grasmere, Staten Island using latitude and longitude values
map_top10 = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, category, name in zip(top10_venues['Venue Latitude'], 
                                           top10_venues['Venue Longitude'], 
                                           top10_venues['Venue Category'], 
                                           top10_venues['Venue']):
    label = '{} - {}'.format(category, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_top10)  
    
map_top10